<a href="https://colab.research.google.com/github/GuptaNavdeep1983/CS767/blob/main/Assignment8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [26]:
import keras
import tensorflow as tf

In [27]:
(X_train_full, y_train_full), (X_test, y_test) = tf.keras.datasets.mnist.load_data()

11493376/11490434 [==============================] - 0s 0us/step


In [30]:
X_train_full[0].shape

(28, 28)

In [38]:

model = keras.models.Sequential()
model.add(keras.layers.Conv2D(filters=64,kernel_size=(3,3), input_shape=(28, 28, 1), activation='relu', padding="SAME"))
model.add(keras.layers.Conv2D(filters=64,kernel_size=(3,3), activation='relu', padding="SAME"))
model.add(keras.layers.MaxPool2D(pool_size=(2,2), strides=2))
model.add(keras.layers.Dropout(rate=0.25))
model.add(keras.layers.Dense(128, activation='relu'))
model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.Dense(10, activation='softmax'))
model.summary()

Model: "sequential_27"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_28 (Conv2D)           (None, 28, 28, 64)        640       
_________________________________________________________________
conv2d_29 (Conv2D)           (None, 28, 28, 64)        36928     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 14, 14, 64)        0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 14, 14, 64)        0         
_________________________________________________________________
dense_1 (Dense)              (None, 14, 14, 128)       8320      
_________________________________________________________________
dropout_4 (Dropout)          (None, 14, 14, 128)       0         
_________________________________________________________________
dense_2 (Dense)              (None, 14, 14, 10)      